In [1]:

from pyphm.datasets.ims import ImsDataLoad
from pathlib import Path
import pandas as pd
import os
import numpy as np
import time
import datetime
import csv

%load_ext autoreload
%autoreload 2

# Full Dataset Test

In [6]:
root_dir = Path.cwd().parent
print(root_dir)
path_data_raw_folder = Path(root_dir / 'data' )
print(path_data_raw_folder)

/home/tim/Documents/PyPHM
/home/tim/Documents/PyPHM/data


In [7]:
ims = ImsDataLoad(path_data_raw_folder, 'ims', download=False)

path_1st_folder = ims.path_1st_folder
path_2nd_folder = ims.path_2nd_folder
path_3rd_folder = ims.path_3rd_folder

file_list = sorted(os.listdir(path_1st_folder))
file_name = file_list[0]

/home/tim/Documents/PyPHM/data/ims


In [8]:
df = ims.load_run_as_df(2)

len file_list: 984
n_jobs: 6


In [8]:
df = ims.load_run_as_df(1)

len file_list: 2156


In [9]:
df.head()

,b1_ch1,b1_ch2,b2_ch3,b2_ch4,id,run,file,time_step
0,-0.049,-0.071,-0.132,-0.010,20,2,2004.02.12.10.32.39,0.000000
1,-0.042,-0.073,-0.007,-0.105,20,2,2004.02.12.10.32.39,0.000049
2,0.015,0.000,0.007,0.000,20,2,2004.02.12.10.32.39,0.000098
3,-0.051,0.020,-0.002,0.100,20,2,2004.02.12.10.32.39,0.000146
4,-0.107,0.010,0.127,0.054,20,2,2004.02.12.10.32.39,0.000195


# Make Ground-Truth Results for Integration Testing

In [2]:
root_dir = Path.cwd().parent
print(root_dir)
path_data_raw_folder = Path(root_dir / 'tests/integration/fixtures' )
print(path_data_raw_folder)

/home/tim/Documents/PyPHM
/home/tim/Documents/PyPHM/tests/integration/fixtures


In [3]:
ims = ImsDataLoad(path_data_raw_folder, download=False)

/home/tim/Documents/PyPHM/tests/integration/fixtures/ims


In [4]:
df = ims.load_run_as_df(1, n_jobs=3)

len file_list: 2
n_jobs: 3


In [5]:
df.head()

,id,run,file,time_step,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8
0,1_0,1,2003.10.22.12.06.24,0.000000,-0.022,-0.039,-0.183,-0.054,-0.105,-0.134,-0.129,-0.142
1,1_0,1,2003.10.22.12.06.24,0.000051,-0.105,-0.017,-0.164,-0.183,-0.049,0.029,-0.115,-0.122
2,1_0,1,2003.10.22.12.06.24,0.000103,-0.183,-0.098,-0.195,-0.125,-0.005,-0.007,-0.171,-0.071
3,1_0,1,2003.10.22.12.06.24,0.000154,-0.178,-0.161,-0.159,-0.178,-0.100,-0.115,-0.112,-0.078
4,1_0,1,2003.10.22.12.06.24,0.000206,-0.208,-0.129,-0.261,-0.098,-0.151,-0.205,-0.063,-0.066


In [6]:
# save the dataframe as ims_truncated_results.csv.gz
df.to_csv(
    root_dir / 'tests/integration/fixtures/ims_truncated_results.csv.gz',
    index=False,
    compression='gzip',
)

In [22]:
df['b1_ch1'].dtype

dtype('float64')

In [23]:
df['b1_ch1'][0]

-0.022

In [24]:
df['b1_ch1'] = df['b1_ch1'].astype(np.float32)

In [25]:
df.head()

,b1_ch1,b1_ch2,b2_ch3,b2_ch4,b3_ch5,b3_ch6,b4_ch7,b4_ch8,id,run,file,time_step
0,-0.022,-0.039,-0.183,-0.054,-0.105,-0.134,-0.129,-0.142,1_0,1,2003.10.22.12.06.24,0.000000
1,-0.105,-0.017,-0.164,-0.183,-0.049,0.029,-0.115,-0.122,1_0,1,2003.10.22.12.06.24,0.000051
2,-0.183,-0.098,-0.195,-0.125,-0.005,-0.007,-0.171,-0.071,1_0,1,2003.10.22.12.06.24,0.000103
3,-0.178,-0.161,-0.159,-0.178,-0.100,-0.115,-0.112,-0.078,1_0,1,2003.10.22.12.06.24,0.000154
4,-0.208,-0.129,-0.261,-0.098,-0.151,-0.205,-0.063,-0.066,1_0,1,2003.10.22.12.06.24,0.000206


# Scratch

In [ ]:
def process_raw_csv(path_run_folder, file_name, sample_index, run_no, sample_freq=20480.0):

    signals_array = np.loadtxt(path_run_folder / file_name, delimiter="\t")

    id_list = [f'{run_no}_{sample_index}'] * len(signals_array)
    run_list = [run_no] * len(signals_array)
    file_list = [file_name] * len(signals_array)
    time_step_array = np.linspace(0.0, len(signals_array) / sample_freq, len(signals_array))

    return signals_array, id_list, run_list, file_list, time_step_array

In [ ]:
file_list = sorted(os.listdir(path_1st_folder))
file_name = file_list[0]

(
    signals_array, 
    id_list, 
    run_list, 
    file_list, 
    time_step_array) = process_raw_csv(path_1st_folder, file_name, 0, 1, sample_freq=20480.0)

In [ ]:
a = np.array(range(0,4))
b = np.array(range(4,8))
c = np.array(range(8,12))
d = np.hstack([a, b, c])
d

In [ ]:
df = ims.load_run_as_df(1)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
run.extend(run)


In [ ]:
len(run)

In [ ]:
a = [signals, signals]
a = np.concatenate(a, axis=0)
a.shape

In [ ]:
df = pd.DataFrame(a, columns=ims.col_1st_names)
df['run'] = run
df.tail()

In [ ]:
run_no = 1
sample_index = 0
id_list = [f'{run_no}_{sample_index}']*10

id_list

In [ ]:
a = np.loadtxt(path_1st_folder / file_name, delimiter="\t")

col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
df = pd.read_csv(path_1st_folder / file_name, sep='\t', names=col_names)
df.head()

In [ ]:
a[1]

In [ ]:
df = ims.load_run_as_dataframe(run_no=1)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
path_1st_folder = ims.path_1st_folder
path_2nd_folder = ims.path_2nd_folder
path_3rd_folder = ims.path_3rd_folder

In [ ]:
file_list = sorted(os.listdir(path_3rd_folder))
file_list[0]

In [ ]:
file_name = file_list[0]
print(file_name)

col_names = ['b1_ch1', 'b1_ch2', 'b2_ch3', 'b2_ch4', 'b3_ch5', 'b3_ch6', 'b4_ch7', 'b4_ch8']
df = pd.read_csv(path_3rd_folder / file_name, sep='\t', names=col_names)
df.head()

In [ ]:
# columns to add
# ['id', 'run', 'start_time_unix', 'date_time', 'unix_time', 'file_name']

df['id'] = "1_" + file_name
df['run'] = 1
df.head()

In [ ]:
def process_raw_csv_ims(path_run_folder, file_name, sample_index, run_no, col_names, sample_freq=20480.0):

    df = pd.read_csv(path_run_folder / file_name, sep='\t', names=col_names)

    df['id'] = "1_" + str(sample_index)
    df['run'] = run_no
    df['file'] = file_name
    df['time'] = np.linspace(0.0, len(df)/sample_freq, len(df))

    return df[['id', 'file', 'run', 'time',] + col_names]

In [ ]:
df1 = process_raw_csv_ims(path_3rd_folder, file_name, 0, 3, col_names, sample_freq=20480.0)
df1.head()

In [ ]:
def create_xy_dataframe():

In [ ]:
df1['time'][2]

In [ ]:
# convert start_time to unix timestamp
start_time_unix = time.mktime(
    datetime.datetime.strptime(file_name, "%Y.%m.%d.%H.%M.%S").timetuple()
)
start_time

In [ ]:
date_nice_format = datetime.datetime.fromtimestamp(start_time).strftime(
    "%Y-%m-%d %H:%M:%S"
)  # reformat date
date_nice_format


In [ ]:
date_list = sorted(os.listdir(path_1st_folder))

In [ ]:
date_list[:10]